In [302]:
using ITensors
using Random
using HDF5
using QuadGK, Roots
using Plots

In [303]:
function loadMPS(path::String; id::String="W")
    """Loads an MPS from a .h5 file. Returns and ITensor MPS."""
        file = path[end-2:end] != ".h5" ? path * ".h5" : path
        f = h5open("$file","r")
        mps = read(f, "$id", MPS)
        close(f)
        return mps
end

loadMPS (generic function with 1 method)

In [304]:
function sliceMPS(W::MPS, class_label::Int)
    """General function to slice the MPS and return the state corresponding to a specific class label."""
    ψ = deepcopy(W)
    decision_idx = findindex(ψ[4], "f(x)")
    decision_state = onehot(decision_idx => (class_label + 1))
    ψ[4] *= decision_state
    normalize!(ψ)

    return ψ
end

sliceMPS (generic function with 1 method)

In [305]:
function get_probability_density(x::Float64, rdm::Matrix)
    """Takes in the 1-site reduced density matrix and 
    returns the probability of a given time series value, x 
    (note, x is before applying encoding, NOT ϕ(x))."""
    # convert time series value to encoded state by applying feature map
    # our complex feature map
    state = [exp(1im * (3π/2) * x) * cospi(0.5 * x), exp(-1im * (3π/2) * x) * sinpi(0.5 * x)]
    return real(state' * rdm * state) # |<x|ρ|x>|
end

function get_normalisation_constant(rdm::Matrix)
    """Get the normalisation constant Z for the pdf"""
    prob_density_wrapper(x) = get_probability_density(x, rdm)
    norm_const, _ = quadgk(prob_density_wrapper, 0, 1) # integrate over data/x domain
    #print(norm_const)
    return norm_const
end

function get_cdf(x::Float64, rdm::Matrix, integral_norm_const::Float64)
    """Get the cumulative distribution function via numerical integration of
    the probability density function.

        Returns cdf evaluated at x where x is the proposed value i.e., F(x)."""

    prob_density_wrapper(x_prime) = (1/integral_norm_const) * get_probability_density(x_prime, rdm)
    cdf_val, _ = quadgk(prob_density_wrapper, 0, x) # intgerate over the domain [-∞, x]
    return cdf_val
end

function sample_state_from_rdm(rdm)
    """Given a 1 site RDM, samples a random value according to the
    conditional distribution encapsulated by the rdm using inverse transform sampling.
    Returns both the sampled value x (the un-feature mapped value) and the feature mapped
    value ϕ(x)."""
    norm_factor = get_normalisation_constant(rdm)
    u = rand() # sample a uniform random value from ~U(0,1)
    cdf_wrapper(x) = get_cdf(x, rdm, norm_factor) - u
    sampled_x = find_zero(cdf_wrapper, (0, 1))
    sampled_state = [exp(1im * (3π/2) * sampled_x) * cospi(0.5 * sampled_x), exp(-1im * (3π/2) * sampled_x) * sinpi(0.5 * sampled_x)]
    return sampled_x, sampled_state
end

sample_state_from_rdm (generic function with 1 method)

In [306]:
mps = loadMPS("/Users/joshua/Documents/QuantumInspiredML/LogLossAlternative/notebooks/iris2.h5")

MPS
[1] ((dim=2|id=172|"S=1/2,Site,n=1"), (dim=2|id=626|"Link,l=1"))
[2] ((dim=2|id=284|"S=1/2,Site,n=2"), (dim=4|id=833|"Link,l=2"), (dim=2|id=626|"Link,l=1"))
[3] ((dim=2|id=918|"S=1/2,Site,n=3"), (dim=6|id=438|"Link,l=3"), (dim=4|id=833|"Link,l=2"))
[4] ((dim=3|id=902|"f(x)"), (dim=2|id=146|"S=1/2,Site,n=4"), (dim=6|id=438|"Link,l=3"))


In [307]:
norm(mps)

0.9999999999999994

In [308]:
state0 = sliceMPS(mps, 0)
mps = deepcopy(state0)
s = siteinds(mps)
orthogonalize!(mps, 1)

MPS
[1] ((dim=2|id=172|"S=1/2,Site,n=1"), (dim=2|id=222|"Link,l=1"))
[2] ((dim=2|id=284|"S=1/2,Site,n=2"), (dim=4|id=705|"Link,l=2"), (dim=2|id=222|"Link,l=1"))
[3] ((dim=2|id=918|"S=1/2,Site,n=3"), (dim=2|id=727|"Link,l=3"), (dim=4|id=705|"Link,l=2"))
[4] ((dim=2|id=146|"S=1/2,Site,n=4"), (dim=2|id=727|"Link,l=3"))


In [309]:
mps

MPS
[1] ((dim=2|id=172|"S=1/2,Site,n=1"), (dim=2|id=222|"Link,l=1"))
[2] ((dim=2|id=284|"S=1/2,Site,n=2"), (dim=4|id=705|"Link,l=2"), (dim=2|id=222|"Link,l=1"))
[3] ((dim=2|id=918|"S=1/2,Site,n=3"), (dim=2|id=727|"Link,l=3"), (dim=4|id=705|"Link,l=2"))
[4] ((dim=2|id=146|"S=1/2,Site,n=4"), (dim=2|id=727|"Link,l=3"))


# Site 1

In [310]:
rdm1 = prime(mps[1], s[1]) * dag(mps[1])

ITensor ord=2 (dim=2|id=172|"S=1/2,Site,n=1")' (dim=2|id=172|"S=1/2,Site,n=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [311]:
rdm1.tensor

Dim 1: (dim=2|id=172|"S=1/2,Site,n=1")'
Dim 2: (dim=2|id=172|"S=1/2,Site,n=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2
 0.23804645328291676 + 0.0im                   …  0.4254524703050361 + 0.007158687684322989im
  0.4254524703050361 - 0.007158687684322989im     0.7619535467170835 + 0.0im

In [312]:
rdm1_mat = matrix(rdm)

2×2 Matrix{ComplexF64}:
 0.238046+0.0im         0.425452+0.00715869im
 0.425452-0.00715869im  0.761954+0.0im

In [313]:
sampled_x, sampled_state = sample_state_from_rdm(rdm1_mat)

(0.48886395137841476, ComplexF64[-0.48128765785576116 + 0.5346504340041781im, -0.4647376557992283 - 0.5162654502260419im])

In [314]:
sampled_x

0.48886395137841476

In [315]:
site_1_state = ITensor(sampled_state, s[1])

ITensor ord=1 (dim=2|id=172|"S=1/2,Site,n=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [316]:
proba_site_1 = get_probability_density(sampled_x, rdm1_mat)

0.43917843946843815

Contract at site 1

In [317]:
mps[1] *= site_1_measured

ITensor ord=1 (dim=2|id=222|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [318]:
mps

MPS
[1] ((dim=2|id=222|"Link,l=1"),)
[2] ((dim=2|id=284|"S=1/2,Site,n=2"), (dim=4|id=705|"Link,l=2"), (dim=2|id=222|"Link,l=1"))
[3] ((dim=2|id=918|"S=1/2,Site,n=3"), (dim=2|id=727|"Link,l=3"), (dim=4|id=705|"Link,l=2"))
[4] ((dim=2|id=146|"S=1/2,Site,n=4"), (dim=2|id=727|"Link,l=3"))


In [319]:
mps[2]*=mps[1]

ITensor ord=2 (dim=2|id=284|"S=1/2,Site,n=2") (dim=4|id=705|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [320]:
mps

MPS
[1] ((dim=2|id=222|"Link,l=1"),)
[2] ((dim=2|id=284|"S=1/2,Site,n=2"), (dim=4|id=705|"Link,l=2"))
[3] ((dim=2|id=918|"S=1/2,Site,n=3"), (dim=2|id=727|"Link,l=3"), (dim=4|id=705|"Link,l=2"))
[4] ((dim=2|id=146|"S=1/2,Site,n=4"), (dim=2|id=727|"Link,l=3"))


In [321]:
mps_updated = MPS(mps[2:end])

MPS
[1] ((dim=2|id=284|"S=1/2,Site,n=2"), (dim=4|id=705|"Link,l=2"))
[2] ((dim=2|id=918|"S=1/2,Site,n=3"), (dim=2|id=727|"Link,l=3"), (dim=4|id=705|"Link,l=2"))
[3] ((dim=2|id=146|"S=1/2,Site,n=4"), (dim=2|id=727|"Link,l=3"))


In [322]:
mps_updated[1] *= 1/sqrt(proba_site_1)

ITensor ord=2 (dim=2|id=284|"S=1/2,Site,n=2") (dim=4|id=705|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [323]:
norm(mps_updated[1])

1.2445832776923553

Site 2

In [324]:
s_updated = siteinds(mps_updated);

In [325]:
rdm2 = prime(mps_updated[1], s_updated[1]) * dag(mps_updated[1]);
abs(tr(rdm2))
rdm2_mat = matrix(rdm2);

In [326]:
sampled_x, sampled_state = sample_state_from_rdm(rdm2_mat);

In [327]:
site_2_state = ITensor(sampled_state, s_updated[1]);
proba_site_2 = get_probability_density(sampled_x, rdm2_mat);

In [328]:
mps_updated[1] *= site_2_state

ITensor ord=1 (dim=4|id=705|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [329]:
mps_updated[2]*=mps_updated[1]

ITensor ord=2 (dim=2|id=918|"S=1/2,Site,n=3") (dim=2|id=727|"Link,l=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [330]:
mps_updated = MPS(mps_updated[2:end])

MPS
[1] ((dim=2|id=918|"S=1/2,Site,n=3"), (dim=2|id=727|"Link,l=3"))
[2] ((dim=2|id=146|"S=1/2,Site,n=4"), (dim=2|id=727|"Link,l=3"))


In [331]:
mps_updated[1] *= 1/sqrt(proba_site_2)

ITensor ord=2 (dim=2|id=918|"S=1/2,Site,n=3") (dim=2|id=727|"Link,l=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [332]:
norm(mps_updated)

1.2172610767918528

----

In [411]:
mps_loaded = loadMPS("/Users/joshua/Documents/QuantumInspiredML/LogLossAlternative/notebooks/iris2.h5")

MPS
[1] ((dim=2|id=172|"S=1/2,Site,n=1"), (dim=2|id=626|"Link,l=1"))
[2] ((dim=2|id=284|"S=1/2,Site,n=2"), (dim=4|id=833|"Link,l=2"), (dim=2|id=626|"Link,l=1"))
[3] ((dim=2|id=918|"S=1/2,Site,n=3"), (dim=6|id=438|"Link,l=3"), (dim=4|id=833|"Link,l=2"))
[4] ((dim=3|id=902|"f(x)"), (dim=2|id=146|"S=1/2,Site,n=4"), (dim=6|id=438|"Link,l=3"))


In [412]:
norm(mps_loaded)

0.9999999999999994

In [413]:
state0 = sliceMPS(mps_loaded, 0)
mps = deepcopy(state0)
s = siteinds(mps)
orthogonalize!(mps, 1)

MPS
[1] ((dim=2|id=172|"S=1/2,Site,n=1"), (dim=2|id=425|"Link,l=1"))
[2] ((dim=2|id=284|"S=1/2,Site,n=2"), (dim=4|id=708|"Link,l=2"), (dim=2|id=425|"Link,l=1"))
[3] ((dim=2|id=918|"S=1/2,Site,n=3"), (dim=2|id=324|"Link,l=3"), (dim=4|id=708|"Link,l=2"))
[4] ((dim=2|id=146|"S=1/2,Site,n=4"), (dim=2|id=324|"Link,l=3"))


In [366]:
A = mps[1]

ITensor ord=2 (dim=2|id=172|"S=1/2,Site,n=1") (dim=2|id=424|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [367]:
rdm1 = prime(A, s[1]) * dag(A)
rdm1_mat = matrix(rdm1)
sampled_x, sampled_state = sample_state_from_rdm(rdm1_mat)
site_1_state = ITensor(sampled_state, s[1])
proba_site_1 = get_probability_density(sampled_x, rdm1_mat)

0.7456362365225491

In [369]:
tr(rdm1_mat)

1.0000000000000002 + 0.0im

In [377]:
Am = A * dag(site_1_state)

ITensor ord=1 (dim=2|id=424|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [378]:
A_new = mps[2] * Am

ITensor ord=2 (dim=2|id=284|"S=1/2,Site,n=2") (dim=4|id=239|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [379]:
A_new *= 1/sqrt(proba_site_1)

ITensor ord=2 (dim=2|id=284|"S=1/2,Site,n=2") (dim=4|id=239|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [383]:
rdm2 = prime(A_new, s[2]) * dag(A_new)
rdm2_mat = matrix(rdm2)
sampled_x2, sampled_state2 = sample_state_from_rdm(rdm2_mat)
site_2_state = ITensor(sampled_state2, s[2])
proba_site_2 = get_probability_density(sampled_x2, rdm2_mat)

0.9273134745222288

In [384]:
real(tr(rdm2_mat))

1.0

In [385]:
Am = A_new * dag(site_2_state)

ITensor ord=1 (dim=4|id=239|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [386]:
A_new = mps[3] * Am

ITensor ord=2 (dim=2|id=918|"S=1/2,Site,n=3") (dim=2|id=678|"Link,l=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [387]:
A_new *= 1/sqrt(proba_site_2)

ITensor ord=2 (dim=2|id=918|"S=1/2,Site,n=3") (dim=2|id=678|"Link,l=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [388]:
rdm3 = prime(A_new, s[3]) * dag(A_new)
rdm3_mat = matrix(rdm3)

2×2 Matrix{ComplexF64}:
  0.750086+0.0im         -0.432859-0.00949351im
 -0.432859+0.00949351im   0.249914+0.0im

In [389]:
tr(rdm3_mat)

1.0000000000000002 + 0.0im

In [390]:
sampled_x3, sampled_state3 = sample_state_from_rdm(rdm3_mat)
site_3_state = ITensor(sampled_state3, s[3])
proba_site_3 = get_probability_density(sampled_x3, rdm3_mat)

0.9316188775403768

In [391]:
Am = A_new * dag(site_3_state)

ITensor ord=1 (dim=2|id=678|"Link,l=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [392]:
A_new = mps[4] * Am

ITensor ord=1 (dim=2|id=146|"S=1/2,Site,n=4")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [393]:
A_new *= 1/sqrt(proba_site_3)

ITensor ord=1 (dim=2|id=146|"S=1/2,Site,n=4")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [397]:
rdm4 = prime(A_new, s[4]) * dag(A_new)
rdm4_mat = matrix(rdm4)
sampled_x4, sampled_state4 = sample_state_from_rdm(rdm4_mat)
site_4_state = ITensor(sampled_state4, s[4])
proba_site_4 = get_probability_density(sampled_x4, rdm4_mat)

0.8682851051068828

In [398]:
real(tr(rdm4_mat))

1.0000000000000004

In [399]:
Am = A_new * dag(site_4_state)

ITensor ord=0
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [406]:
println(sampled_x)
println(sampled_x2)
println(sampled_x3)
println(sampled_x4)

0.8243705897047662
0.4652274718457367
0.39357749219659394
0.3320561044280189


In [447]:
function sample_mps_with_contractions(label_mps::MPS)
    """Using the perfect sampling approach described in Vidal et al.'s paper
    on sampling from Unitaary Tensor Networks."""
    # do initial checks on the mps
    @assert isapprox(norm(label_mps), 1.0) "WARNING: MPS NOT NORMALISED!"
    # make a copy of the mps
    mps = deepcopy(label_mps)
    s = siteinds(mps)
    # put the mps into right canonical form - orthogonality center is set to the first site
    orthogonalize!(mps, 1)
    # create storage for samples
    x_samps = Vector{Float64}(undef, length(mps))
    #println(length(x_samps))

    # set A to the first mps site
    A = mps[1]

    for i in 1:length(mps)

        # get the reduced density matrix at site i
        rdm = prime(A, s[i]) * dag(A)
        # convert to matrix type
        rdm_matrix = matrix(rdm)
        # sample a state from the rdm using inverse transform sampling
        sampled_x, sampled_state = sample_state_from_rdm(rdm_matrix)
        x_samps[i] = sampled_x
        if i != length(mps)
            sampled_state_as_ITensor = ITensor(sampled_state, s[i])
            # get the probability of the state for normalising the next site
            proba_state = get_probability_density(sampled_x, rdm_matrix)
            println("Prob of sampled state: $proba_state")
            # check that the trace of the rdm is equal to one
            sampled_x, sampled_state = sample_state_from_rdm(rdm_matrix)
            println("Trace of ρ$i: $(real(tr(rdm_matrix)))")
            # make the measurment of the site
            Am = A * dag(sampled_state_as_ITensor)
            # absorb into the next site
            A_new = mps[(i+1)] * Am
            # normalise by the probability
            A_new *= 1/sqrt(proba_state)
            # set A to A_new
            A = A_new
        end
    end

    return x_samps

end

sample_mps_with_contractions (generic function with 1 method)

In [456]:
out = sample_mps_with_contractions(mps)

Prob of sampled state: 0.6736585248342197
Trace of ρ1: 1.0000000000000002
Prob of sampled state: 0.46599076680053353
Trace of ρ2: 1.0000000000000004
Prob of sampled state: 0.6825151465517019
Trace of ρ3: 0.9999999999999998


4-element Vector{Float64}:
 0.8996155414766933
 0.03789937958033243
 0.0509048476589282
 0.055503191778310125